# Excel Reporting

## Overview
Generate professional Excel financial statements -- balance sheets, income statements, cash-flow statements, and multi-year dashboards -- from simulation results.

- **Prerequisites**: [reporting/01_table_generation](01_table_generation.ipynb)
- **Estimated runtime**: 1-2 minutes
- **Audience**: [Practitioner]

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

from ergodic_insurance.config import ManufacturerConfig, ExcelReportConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator
from ergodic_insurance.insurance import InsurancePolicy, InsuranceLayer
from ergodic_insurance.excel_reporter import ExcelReporter
from ergodic_insurance.financial_statements import (
    FinancialStatementGenerator, FinancialStatementConfig,
)

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Reproducibility
SEED = 42
np.random.seed(SEED)

## 1. Simulate 10 Years of Operations

Create a manufacturer, attach insurance, and run a 10-year trajectory.

In [ ]:
manufacturer_config = ManufacturerConfig(
    initial_assets=10_000_000,
    asset_turnover_ratio=0.5,
    base_operating_margin=0.08,
    tax_rate=0.25,
    retention_ratio=0.7,
)
manufacturer = WidgetManufacturer(manufacturer_config)

claim_generator = ManufacturingLossGenerator.create_simple(
    frequency=5.3, severity_mean=500_000,
    severity_std=750_000, seed=SEED,
)

insurance_layer = InsuranceLayer(
    attachment_point=100_000, limit=5_000_000, rate=0.02,
)
insurance = InsurancePolicy(layers=[insurance_layer], deductible=100_000)

simulation_years = 10
all_claims = claim_generator.generate_claims(simulation_years)
claims_by_year = {}
for c in all_claims:
    claims_by_year.setdefault(c.year, []).append(c.amount)

for year in range(simulation_years):
    total = sum(claims_by_year.get(year, []))
    if total > 0:
        co_pay, ins_pay = insurance.process_claim(total)
    else:
        co_pay, ins_pay = 0, 0
    premium = insurance.calculate_premium()
    manufacturer.step(letter_of_credit_rate=0.015, growth_rate=0.03)
    manufacturer.assets -= premium + co_pay
    manufacturer.equity -= premium + co_pay

print(f"Final equity: ${manufacturer.equity:,.0f}")

## 2. Financial Statements

In [ ]:
stmt_config = FinancialStatementConfig(
    currency_symbol="$", decimal_places=0, include_yoy_change=True,
)
manufacturer_data = {
    "metrics_history": manufacturer.metrics_history,
    "initial_assets": manufacturer.config.initial_assets,
    "config": manufacturer.config,
}
stmt_gen = FinancialStatementGenerator(manufacturer_data=manufacturer_data, config=stmt_config)

last_year = stmt_gen.years_available - 1
balance_sheet = stmt_gen.generate_balance_sheet(last_year)
income_stmt   = stmt_gen.generate_income_statement(last_year)
cash_flow     = stmt_gen.generate_cash_flow_statement(last_year, method="indirect")

print(f"Balance Sheet  : {len(balance_sheet)} rows")
print(f"Income Stmt    : {len(income_stmt)} rows")
print(f"Cash Flow Stmt : {len(cash_flow)} rows")

## 3. Excel Export

In [ ]:
output_dir = Path("./excel_reports")
output_dir.mkdir(exist_ok=True)

basic_file = output_dir / "basic_financial_statements.xlsx"
with pd.ExcelWriter(basic_file, engine="openpyxl") as writer:
    balance_sheet.to_excel(writer, sheet_name="Balance Sheet", index=False)
    income_stmt.to_excel(writer, sheet_name="Income Statement", index=False)
    cash_flow.to_excel(writer, sheet_name="Cash Flow", index=False)

print(f"Basic report saved to: {basic_file}")

## 4. Multi-Year Summary

In [ ]:
rows = []
for yr in range(stmt_gen.years_available):
    m = manufacturer.metrics_history[yr]
    rows.append({
        "Year": yr,
        "Assets": m.get("assets", 0),
        "Equity": m.get("equity", 0),
        "Revenue": m.get("revenue", 0),
        "Net Income": m.get("net_income", 0),
        "ROE %": m.get("roe", 0) * 100,
        "ROA %": m.get("roa", 0) * 100,
    })
multi_year = pd.DataFrame(rows)
multi_year["Revenue Growth %"] = multi_year["Revenue"].pct_change() * 100

display(multi_year)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 9))

axes[0, 0].plot(multi_year["Year"], multi_year["Assets"], "o-", label="Assets")
axes[0, 0].plot(multi_year["Year"], multi_year["Equity"], "s-", label="Equity")
axes[0, 0].set_title("Assets & Equity")
axes[0, 0].legend()

axes[0, 1].plot(multi_year["Year"], multi_year["Revenue"], "o-", label="Revenue")
axes[0, 1].plot(multi_year["Year"], multi_year["Net Income"], "s-", label="Net Income")
axes[0, 1].set_title("Revenue & Net Income")
axes[0, 1].legend()

axes[1, 0].plot(multi_year["Year"], multi_year["ROE %"], "o-", label="ROE")
axes[1, 0].plot(multi_year["Year"], multi_year["ROA %"], "s-", label="ROA")
axes[1, 0].set_title("Return Ratios")
axes[1, 0].legend()

axes[1, 1].bar(multi_year["Year"][1:], multi_year["Revenue Growth %"][1:], alpha=0.7)
axes[1, 1].set_title("Revenue Growth %")

for ax in axes.flat:
    ax.set_xlabel("Year")
    ax.grid(True, alpha=0.3)

plt.suptitle("Multi-Year Financial Summary", fontweight="bold")
plt.tight_layout()
plt.show()

## Key Takeaways

- `FinancialStatementGenerator` produces balance sheets, income statements, and cash-flow statements from a manufacturer's metrics history.
- `ExcelReporter` adds professional formatting, conditional colouring, and dashboard sheets.
- Multi-year summaries with growth-rate calculations help track trajectory health.

## Next Steps

- [reporting/03_report_generation](03_report_generation.ipynb) -- automated executive and technical reports
- [reporting/01_table_generation](01_table_generation.ipynb) -- table formatting utilities